# python-binexport: Quick Start

## Introduction

The sample to work on has been extracted from the firmware of a edimax router (BR6478AC V2) and it is known to be vulnerable to CVE-2023-49351.
It can be downloaded below.

It's a MIPS ELF executable that acts as a HTTP server. We are interested in finding all the functions that call a potentially unsafe primitive function (like `strcpy`).

Let's use python-binexport to list them.

## I. Loading the program

If the program has not been exported with BinExport, it can be exported with:

In [ ]:
from binexport import ProgramBinExport

program = ProgramBinExport.from_binary_file("./webs")

Otherwise it can be directly loaded with:

In [2]:
from binexport import ProgramBinExport

program = ProgramBinExport("./webs.BinExport")

## II. Listing interesting functions

We are interested in listing all the functions that call a potentially vulnerable function, like `strcpy` for example.
This can be achieved by using the [FunctionBinExport.parents](https://diffing.quarkslab.com/exporter/binexport.html#binexport.function.FunctionBinExport.parents) API.

In [3]:
# Get the vulnerable function
strcpy = program.fun_names["strcpy"]

targets = strcpy.parents
print("\n".join(map(lambda f: f.name, targets)))

formiNICWdsEncrypt
dhcpClientList
CharFilter4
formWpsStart
formQoS
wlSiteSurveyTbl
getInfo
formWIRESch
get_dir
formiNICSiteSurvey
formiNICWpsStart
wliNICSiteSurveyonlyTbl
websFormDefine
formFilter
getiNICIndex
formiNICbasic
formVPNuser
wlSurveyOnlyTbl
formiNICEncrypt
formEZQoS
formTriggerPort
websAspDefine
formWlAc
CharFilter2
formWlSiteSurvey
formWdsEncrypt
WIRESchList
ACPCList
USBFolderSelect
formPortFw
sub_434E40
wliNICSiteSurveyTbl
wispSiteSurveyTbl5G
CharFilter3
formDNSProxyrules
formUSBAccount
CharFilter5
getIndex
CharFilter0
formWlEncrypt
formWlbasic
formiNICAc
CharFilter6
setWAN
getiNICInfo
wispSiteSurveyTbl
getInAddr
apmib_set
getWlSiteSurveyRequest
formSaveText
formUSBFolder
apmib_get
formVirtualSv


## III. Find interesting paths in the CG

Now what if we wanted to show the path in the CG (Call Graph) that leads to the execution of one of those functions from `main`?

In [4]:
from binexport import FunctionBinExport


def dfs(f: FunctionBinExport, path: list[FunctionBinExport]):
    if f.name == "main":
        print(" > ".join(map(lambda x: x.name, path[::-1])))
        return
    
    for caller in f.parents:
        path.append(caller)
        dfs(caller, path)
        path.pop(-1)

for target in targets:
    dfs(target, [strcpy, target])

main > websAspInit > formsDefine > formiNICWdsEncrypt > strcpy
main > websAspInit > formsDefine > dhcpClientList > strcpy
main > websAspInit > formsDefine > virtualSvList > CharFilter4 > strcpy
main > websAspInit > formsDefine > wlAcList > CharFilter4 > strcpy
main > websAspInit > formsDefine > portFwList > CharFilter4 > strcpy
main > websAspInit > formsDefine > wliNICAcList > CharFilter4 > strcpy
main > websAspInit > formsDefine > macFilterList > CharFilter4 > strcpy
main > websAspInit > formsDefine > ACPCList > CharFilter4 > strcpy
main > websAspInit > formsDefine > triggerPortList > CharFilter4 > strcpy
main > websAspInit > formsDefine > formWpsStart > strcpy
main > websAspInit > formsDefine > formQoS > strcpy
main > websAspInit > formsDefine > wlSiteSurveyTbl > strcpy
main > websAspInit > formsDefine > getInfo > strcpy
main > websAspInit > formsDefine > formWIRESch > strcpy
main > loop > process_requests > read_header > process_header_end > init_get > get_dir > strcpy
main > loop >

This can gives us an idea of which functions to look at when searching for a vulnerability.